In [66]:
import os
import pandas as pd
if os.path.isfile(r'E:\source\pygame\Nado Game\pyqt5\자재관리\입고대장.xlsx'):
    LE =  [
        'E:/source/pygame/Nado Game/pyqt5/자재관리/입고대장.xlsx',
        'E:/source/pygame/Nado Game/pyqt5/자재관리/사용대장.xlsx',
        'E:/source/pygame/Nado Game/pyqt5/자재관리/동호대장.xlsx',
        ]
else: 
    LE =  [
        'C:/source/pygame/Nado Game/pyqt5/자재관리/입고대장.xlsx',
        'C:/source/pygame/Nado Game/pyqt5/자재관리/사용대장.xlsx',
        'C:/source/pygame/Nado Game/pyqt5/자재관리/동호대장.xlsx',
        ]
def out_df():
    file = LE[1] #r'C:\source\pygame\Nado Game\pyqt5\자재관리\사용대장.xlsx'
    with pd.ExcelFile(file) as f:
        df = pd.read_excel(f,skiprows=0)
    return df
def in_df():
    file = LE[0] #r'C:\source\pygame\Nado Game\pyqt5\자재관리\입고대장.xlsx'
    with pd.ExcelFile(file) as f:
        df = pd.read_excel(f,skiprows=0)
    return df


In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    """여러 데이터프레임 비교가 쉽게 옆쪽으로 표시한다"""
    html_str=''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)
dfo =out_df()
dfi =in_df()
display_side_by_side(dfi,dfo)

In [47]:
from IPython.display import display_html
def display_side_by_side(*args):
    """여러 데이터프레임 비교가 쉽게 옆쪽으로 표시한다"""
    html_str=''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)

In [ ]:
dfo =out_df()
dfi =in_df()
dfiq = dfi.query('(품명 == "LED 전구") and (규격 == "11w")')
dfiqs = dfiq.sort_values(by=['입고일'])
#dfiqs['sum'] = dfiqs.cumsum([['입고수량']])
dfiqs['cumsum'] = dfiqs.groupby((dfiqs[[규격]]).cumsum()).입고수량.cumsum()

In [ ]:
dfo =out_df()
dfoq = dfo.query('(품명 == "LED 전구") and (규격 == "11w")')
dfoqs = dfoq.sort_values(by=['사용일'])
dfoqs

In [26]:
#df['cumsum'] = df.groupby((df.NUM == 10).cumsum()).ENERGY.cumsum()
#출처: https://note.espriter.net/1339 [espriter:티스토리]
dfo =in_df()
dfo[['구입금액', '단가']].astype('str')
dfo.groupby(['품명', '규격','입고일']).sum(['입고수량']).groupby(level=0).cumsum().reset_index()


,품명,규격,입고일,입고수량,구입금액,단가
0,LED 전구,11w,2022-05-03,1,1,1
1,LED 전구,11w,2022-05-10,24,542122,23571
2,LED 전구,11w,2022-05-16,47,1084243,47141
3,LED 전구,11w,2022-05-23,70,1626364,70711
4,LED 전구,65w,2022-05-24,90,1676364,73211
5,LED 전구,EL-11W(삼파장),2022-05-03,91,1676365,73212
6,LED 전구,EL-20W(삼파장),2022-05-03,93,1676367,73214
7,LED 전등,센서등 13w,2022-05-21,1,250000,250000
8,LED 전등,센서모듈 1등용,2022-05-03,2,250001,250001
9,가스,가스탐지기,2022-05-03,3,3,3


In [83]:
# def cum_in_calc(self,df)
dfo =in_df()
items = dfo['품명'].unique()
#spec = dfo['규격'].unique() 
df_list = []
for i in items:
    dfo_s = dfo[(dfo['품명'] == i)]
    spec = dfo_s['규격'].unique()
    for s in spec:
        
        dfo_sel = dfo[(dfo['품명'] == i) & (dfo['규격'] == s)].copy()# & dfo['규격'] == '11w')]
        dfo_temp = dfo_sel.sort_values(by=['입고일']).copy()
        dfo_copy = dfo_temp[['입고수량']].copy()
        dfo_copy['품목 누계'] = dfo_copy.cumsum()
        dfo_temp['품목 누계']=dfo_copy['품목 누계']
        df_list.append(dfo_temp)
        
df_list[1]
df_con = df_list[0]
for l in df_list[1:]:
    df_con = pd.concat([df_con, l])
df_con.
df_con.sort_values(by=['입고일'])   
df_con.columns.tolist()
df_con[['입고일', '품명', '규격', '입고수량', '품목 누계', '구입금액', '단가', '구입업체', '비고' ]]
#display_side_by_side(dfo_temp, dfo_copy)

SyntaxError: invalid syntax (<ipython-input-83-bd499fae8625>, line 22)

In [35]:
dfo

,입고일,품명,규격,입고수량,구입금액,단가,구입업체,비고
0,2022-05-03,LED 전구,11w,1,1,1,인터넷 쇼핑몰,*
1,2022-05-03,LED 전구,EL-11W(삼파장),1,1,1,인터넷 쇼핑몰,*
2,2022-05-03,LED 전구,EL-20W(삼파장),1,1,1,인터넷 쇼핑몰,*
3,2022-05-03,LED 전구,EL-20W(삼파장),1,1,1,인터넷 쇼핑몰,*
4,2022-05-03,형광등,FDX-18W 램프,1,1,1,인터넷 쇼핑몰,*
...,...,...,...,...,...,...,...,...
69,2022-05-03,할로겐 등,할로겐 50W 안정기,1,1,1,인터넷 쇼핑몰,*
70,2022-05-23,LED 전구,11w,23,542121,23570,111,111
71,2022-05-16,LED 전구,11w,23,542121,23570,111,111
72,2022-05-10,LED 전구,11w,23,542121,23570,111,111
